In [2]:
# Установка необходимых библиотек
import sys
!pip install pandas
!pip install tqdm
!pip install pysubs2

In [1]:
# импорт зависимостей
import re
from pathlib import Path
import pysubs2
import os
import pandas as pd
import IPython.display as ipd
from tqdm.notebook import tqdm
import csv
import fnmatch
import soundfile as sf
import numpy as np

In [2]:
class TextInspector:    
    def __init__(self, phrases, full_check=True):
        self.clone = {'a': 'а','c': 'с','e': 'е','o': 'о','p': 'р','x': 'х','y': 'у','A': 'А','B': 'В','C': 'С','E': 'Е','H': 'Н','K':'К','M': 'М','O': 'О','P': 'Р','T': 'Т','X': 'Х'}
        self.total_words = 0
        self.words_without_stress = []
        self.words_with_many_stress = []        
        self.phrases = phrases
        
        if full_check:
            self.check_strange_stress()
#             self.check_latin_symbols()
            self.check_numbers_symbols()
            self.check_last_phrase_symbol()
            
        
    def get_symbols(self):
        symbols_set = set()
        output_repr = ''
        
        for phrase in self.phrases:
            symbols_set.update(phrase)
            
        for i,symbol in enumerate(sorted(list(symbols_set))):
            if i%10==0:
                output_repr+='\n'
            
            output_repr += f" ({symbol} {ord(symbol)}) \t"
        
        print(output_repr)
                     
    def get_words_stat(self):
        for i,phrase in enumerate(self.phrases):
            for word in re.findall(r'([а-яёА-ЯЁ\'-]+)', phrase):
                stress_count = self.stress_count(word)
                if stress_count==0 and self.vowel_count(word)>1:
                    self.words_without_stress.append((i+1, word))
                elif stress_count>1:
                    self.words_with_many_stress.append((i+1, word))
                    
                self.total_words+=1
                
        print(f"Total words: {self.total_words}\nWords without stress: {len(self.words_without_stress), self.words_without_stress}\nWords with many stress: {len(self.words_with_many_stress), self.words_with_many_stress}", end='\n\n')
            
    def check_strange_stress(self):
        for i,phrase in enumerate(self.phrases):
            flag = False
            for index in self._accents_indices(phrase):
                if index == 0 or phrase[index-1] not in "уеыаоэяиюёУЕЫАОЭЯИЮЁ'":
                    flag = True
            
            if flag:
                print(f"Bad stress in {i+1} phrase : {phrase}")
    
    def check_numbers_symbols(self):
        for i,phrase in enumerate(self.phrases):
            result = re.findall(r'([0-9]+)', phrase)            
            if len(result)>0:
                print(f"Numbers symbols {result} in {i+1} phrase : {phrase}")
                
    def check_latin_symbols(self):
        for i,phrase in enumerate(self.phrases):
            result = re.findall(r'([a-zA-Z]+)', phrase)            
            if len(result)>0:
                print(f"Latin symbols {result} in {i+1} phrase : {phrase}")
                
    def check_last_phrase_symbol(self):
        indices = []
        for i,phrase in enumerate(self.phrases):
            if phrase != "":
                if phrase[-1] not in '.!?"':
                    indices.append(i+1)
            
        if len(indices)>0:
            print(f"Bad end of phrase in: {indices}")
            
    def _accents_indices(self,text):
        indices = []
        previous_len = 0
        i = text.find("'")

        while i>-1:
            indices.append(i+previous_len)
            previous_len += i+1
            i = text[previous_len:].find("'")

        return indices
        
    def put_stress_one_vowel_words(self):
         for i,phrase in enumerate(self.phrases):
            parts = phrase.split()
            for j,part in enumerate(parts):
                stress_count = self.stress_count(part)
                vowels = self.vowels(part)
                if len(vowels)==1 and stress_count==0:
                    parts[j] = part.replace(vowels, vowels+"'")
                    
            self.phrases[i] = ' '.join(parts).strip()

    def replace_separate_latins(self):
        for i,phrase in enumerate(self.phrases):
            parts = phrase.split()
            for j,part in enumerate(parts):
                latins = re.findall(r'([a-zA-Z]+)', part) 
                if len(latins)==1 and latins[0] in self.clone:
                    parts[j] = part.replace(latins[0], self.clone[latins[0]])
            
            self.phrases[i] = ' '.join(parts).strip()
            
    def replace(self, old_char, new_char):
        for i,phrase in enumerate(self.phrases):
            self.phrases[i] = phrase.replace(old_char, new_char)
            
    def show_abbr_and_latin(self):
        for i,phrase in enumerate(self.phrases):
            result = re.findall(r'([^\s]+\[+(?:[^\s])+\])', phrase)
            
            if len(result)>0:
                print(f"Abbr and latin disclosure in {i+1}: {result}")
            
    def find_phrase_with_str(self, string):
        return [(i+1, phrase) for i,phrase in enumerate(self.phrases) if string in phrase]
    
    def vowels(self, word):
        return re.sub(r'([^уеыаоэяиюёУЕЫАОЭЯИЮЁ]+)', "", word)
    
    def vowel_count(self, text):
        return len(self.vowels(text))
    
    def stress_count(self, text):
        return text.count("'")

In [3]:
for ass_file in Path(r"C:\Users\yamilova_da\Desktop\Дроздов_Оля").glob("*.ass"):
    print(ass_file.stem)
    try:
        subs = pysubs2.load(ass_file)
    #     with open(ass_file, "r", encoding="utf-8") as subs:
    except AttributeError:
        print("0"+ass_file.stem)
        next
    phrases = [line.text for line in subs]
#         phrases = [line.replace('\n', '') for line in subs]

    test = TextInspector(phrases)
    
        # заменяем неугодные символы
    test.replace("…", "...")
    test.replace(chr(8230), "")
    test.replace(chr(171), '"')
    test.replace(chr(187), '"')
    test.replace(chr(8212),'-')
    test.replace(chr(8211),'-')
    
    test.words_without_stress
    test.replace_separate_latins()
#     test.put_stress_one_vowel_words()
    
    test.get_symbols()
    test.get_words_stat()
    test.words_without_stress
    for line,phrase in zip(subs,test.phrases):
        
        line.text = phrase

    subs.save(ass_file)

2018_04_14_to_record_11_master_1

 (  32) 	 (! 33) 	 (" 34) 	 (' 39) 	 (, 44) 	 (- 45) 	 (. 46) 	 (: 58) 	 (; 59) 	 (? 63) 	
 (а 1072) 	 (б 1073) 	 (в 1074) 	 (г 1075) 	 (д 1076) 	 (е 1077) 	 (ж 1078) 	 (з 1079) 	 (и 1080) 	 (й 1081) 	
 (к 1082) 	 (л 1083) 	 (м 1084) 	 (н 1085) 	 (о 1086) 	 (п 1087) 	 (р 1088) 	 (с 1089) 	 (т 1090) 	 (у 1091) 	
 (ф 1092) 	 (х 1093) 	 (ц 1094) 	 (ч 1095) 	 (ш 1096) 	 (щ 1097) 	 (ъ 1098) 	 (ы 1099) 	 (ь 1100) 	 (э 1101) 	
 (ю 1102) 	 (я 1103) 	 (ё 1105) 	
Total words: 7112
Words without stress: (10, [(47, 'уши'), (103, 'уши'), (132, 'полу'), (317, 'руки'), (767, 'было'), (819, 'было'), (850, 'руки'), (1148, 'руку'), (1154, 'руку'), (1158, 'было')])
Words with many stress: (8, [(75, "о'чень-о'чень"), (87, "никогда'-никогда'"), (90, "о'чень-о'чень"), (153, "до'чки-ма'тери"), (457, "та'к-та'к"), (499, "туда'-сюда'"), (1057, "до'чки-ма'тери"), (1061, "далеко'-далеко'")])

2018_04_19_to_record_12_master_1
Bad end of phrase in: [255]

 (  32) 	 (! 33) 	 (" 34)

Words with many stress: (20, [(13, "ки''берспортсме'на"), (15, "туда'-сюда'"), (19, "пятью''десятью'"), (28, "Четырё''хкра'тный"), (28, "ки''нопре'мии"), (41, "ки''берспортсме'нами"), (54, "А'нны-Генрие'тты"), (84, "а''нтигеро'й"), (91, "со''циологи'ческое"), (91, "а''нтропологи'ческое"), (94, "ки''берспортсме'ны"), (97, "кро''вожа'дный"), (113, "по''лфи'льма"), (135, "Нью'-Йо'рке"), (143, "програ'ммы-тренажё'ры"), (155, "са''моидентифика'ции"), (159, "фо''тоисто'рия"), (183, "спе''цэффе'кты"), (187, "фра'нко-кана'дский"), (187, "ки''норежиссё'р")])

Review_12_master_1

 (  32) 	 (" 34) 	 (' 39) 	 (( 40) 	 () 41) 	 (, 44) 	 (- 45) 	 (. 46) 	 (: 58) 	 (; 59) 	
 (? 63) 	 (А 1040) 	 (Б 1041) 	 (В 1042) 	 (Г 1043) 	 (Д 1044) 	 (Е 1045) 	 (З 1047) 	 (И 1048) 	 (К 1050) 	
 (М 1052) 	 (Н 1053) 	 (О 1054) 	 (П 1055) 	 (Р 1056) 	 (С 1057) 	 (Т 1058) 	 (У 1059) 	 (Ф 1060) 	 (Х 1061) 	
 (Ч 1063) 	 (Э 1069) 	 (Я 1071) 	 (а 1072) 	 (б 1073) 	 (в 1074) 	 (г 1075) 	 (д 1076) 	 (е 1077) 	 (ж 1078) 	
 

 (щ 1097) 	 (ъ 1098) 	 (ы 1099) 	 (ь 1100) 	 (э 1101) 	 (ю 1102) 	 (я 1103) 	 (ё 1105) 	
Total words: 1834
Words without stress: (1, [(116, 'лиаз')])
Words with many stress: (24, [(18, "ари'шкой-труси'шкой"), (43, "ба'тюшки-свя'ты"), (65, "са''монаде'янный"), (65, "а''втолюби'тель"), (71, "по''лго'да"), (75, "са''моутверди'ться"), (81, "эле''ктропо'езд"), (87, "собра'тья-поезда'"), (88, "ле''сопоса'дках"), (100, "тё'мно-стальну'ю"), (101, "зелё'ные-зелё'ные"), (116, "тё'мно-зелё'ным"), (132, "трё''хла'пый"), (140, "желе''знодоро'жном"), (149, "скве''рносло'в"), (157, "стре'лка-помо'щница"), (158, "желе''знодоро'жном"), (160, "по''лчаса'"), (170, "са''моутвержде'нии"), (174, "желе''знодоро'жные"), (188, "са''мочу'вствии"), (188, "трё''хэта'жно"), (192, "то'лько-то'лько"), (200, "сни'зу-вве'рх")])

tender_3_03_master_1

 (  32) 	 (! 33) 	 (" 34) 	 (' 39) 	 (( 40) 	 () 41) 	 (, 44) 	 (- 45) 	 (. 46) 	 (: 58) 	
 (а 1072) 	 (б 1073) 	 (в 1074) 	 (г 1075) 	 (д 1076) 	 (е 1077) 	 (ж 1078) 	 (

In [36]:
test.get_symbols() # посмотрим на все символы 


 (  32) 	 (! 33) 	 (" 34) 	 (' 39) 	 (( 40) 	 () 41) 	 (, 44) 	 (- 45) 	 (. 46) 	 (: 58) 	
 (? 63) 	 (E 69) 	 (F 70) 	 (N 78) 	 ([ 91) 	 (] 93) 	 (a 97) 	 (b 98) 	 (c 99) 	 (d 100) 	
 (e 101) 	 (f 102) 	 (i 105) 	 (l 108) 	 (m 109) 	 (n 110) 	 (o 111) 	 (p 112) 	 (r 114) 	 (s 115) 	
 (t 116) 	 (u 117) 	 (v 118) 	 (y 121) 	 (z 122) 	 (ü 252) 	 (А 1040) 	 (К 1050) 	 (М 1052) 	 (Н 1053) 	
 (О 1054) 	 (Ф 1060) 	 (а 1072) 	 (б 1073) 	 (в 1074) 	 (г 1075) 	 (д 1076) 	 (е 1077) 	 (ж 1078) 	 (з 1079) 	
 (и 1080) 	 (й 1081) 	 (к 1082) 	 (л 1083) 	 (м 1084) 	 (н 1085) 	 (о 1086) 	 (п 1087) 	 (р 1088) 	 (с 1089) 	
 (т 1090) 	 (у 1091) 	 (ф 1092) 	 (х 1093) 	 (ц 1094) 	 (ч 1095) 	 (ш 1096) 	 (щ 1097) 	 (ъ 1098) 	 (ы 1099) 	
 (ь 1100) 	 (э 1101) 	 (ю 1102) 	 (я 1103) 	 (ё 1105) 	


In [6]:
test.get_words_stat()

Total words: 4572
Words without stress: (28, [(1, 'Минэнерго'), (25, 'генплан'), (31, 'Минтранса'), (33, 'Минкультуры'), (45, 'Роскачество'), (53, 'Роскачества'), (91, 'Лендоке'), (98, 'было'), (102, 'БИОС-'), (106, 'Ростуризма'), (127, 'Госдуме'), (150, 'было'), (154, 'жилтоварищество'), (190, 'ОАЭ'), (1, 'Минэнерго'), (25, 'генплан'), (31, 'Минтранса'), (33, 'Минкультуры'), (45, 'Роскачество'), (53, 'Роскачества'), (91, 'Лендоке'), (98, 'было'), (102, 'БИОС-'), (106, 'Ростуризма'), (127, 'Госдуме'), (150, 'было'), (154, 'жилтоварищество'), (190, 'ОАЭ')])
Words with many stress: (140, [(1, "ми''н-эне'рго"), (9, "трё''хдне'вного"), (14, "ми''крорайо'не"), (16, "шести''деся'тые"), (18, "интерне'т-се'рвисов"), (23, "соверше''нноле'тним"), (25, "ге''н-пла'н"), (28, "Ша'рм-эль-Ше'йх"), (28, "а''виакомпа'нии"), (29, "зло''ка'чественных"), (29, "но''вообразова'ний"), (31, "ми''н-тра'нса"), (33, "ми''н-культу'ры"), (34, "вре''мяпровожде'ния"), (35, "ту''р-маршру'ты"), (36, "Го''сду'ма"), (37,

In [13]:
test.words_without_stress # слова без ударений с указанием номеров предложений для удобства редактирования файлу источнике

[(20, 'госуслуги'),
 (22, 'руку'),
 (62, 'лже-МФО'),
 (92, 'госуслуги'),
 (139, 'бизнесинвест')]

In [14]:
test.check_numbers_symbols() # проверка на наличие цифр в корпусе

In [15]:
test.check_strange_stress() # проверка на некорректные ударения

In [13]:
test.show_abbr_and_latin()

In [16]:
test.replace_separate_latins() # заменем всевозможные варианты случайной латиницы

In [17]:
test.check_latin_symbols() # смотрим на латиницу в корпусе

Latin symbols ['industrial', 'Einst', 'rzende', 'Neubauten'] in 36 phrase : и' во'т они' постепе'нно вхо'дят в ри'тм и' начина'ют оття'гиваться в по'лный ро'ст, получа'ется тако'й industrial[инда'стриал], ти'па Einstürzende[айшту'нценде] Neubauten[нойба'утэн].
Latin symbols ['Fraud', 'prevention', 'service'] in 137 phrase : мы' испо'льзуем Fraud[фро'д] prevention[прывэ'ншн] service[сё'рвич] бюро' креди'тных исто'рий эквифа'кс.
Latin symbols ['money', 'funny'] in 154 phrase : "в "money[ма'ни] funny[фа'нни]" тако'й спи'сок ведё'тся", - добавля'ет шу'стов.


In [66]:
test.put_stress_one_vowel_words() # ударение одногласных слов

Код ниже позволяет найти текстовые дубликаты в корпусах

In [20]:
# Вспомогательные функции

def find_all_files(path, expansion='.ass'):
    lst = []
    for root, dirnames, filenames in os.walk(path, followlinks=True):
        for filename in fnmatch.filter(filenames, '*' + expansion):
            lst.append(os.path.join(root, filename))
    return lst

def find_text_duplicates(speaker_path, output_path=None):
    ass = find_all_files(speaker_path, '.ass')
    all_dialogues = []

    for idx, file_ass in enumerate(tqdm(ass, desc='parse ass files')):
        subs = pysubs2.load(file_ass)
        audio_name = subs.aegisub_project['Audio File']
        ass_file = Path(file_ass).relative_to(speaker_path)

        for pos, line in enumerate(subs):
            if line.type == "Dialogue":
                start = pysubs2.substation.ms_to_timestamp(line.start)
                end = pysubs2.substation.ms_to_timestamp(line.end)
                all_dialogues.append((ass_file, audio_name, pos, line.plaintext, start, end))

    df = pd.DataFrame.from_records(all_dialogues, columns=['ass_file', 'audio_file', 'pos', 'text', 'start', 'end'])
    df['text_len'] = df['text'].str.len()

    # find text duplicates
    texts = df['text'].apply(lambda x: x.lower())
    duplicates_df = df[texts.isin(texts[texts.duplicated()])]
    duplicates_df = duplicates_df.sort_values(by=['text_len', 'text'])
    duplicates_df = duplicates_df.reset_index()
    duplicates_df = duplicates_df.drop(columns=['index','text_len'])
    print(f'found {duplicates_df.shape[0]} text duplicates.')

    # save table
    if output_path and duplicates_df.shape[0] > 0:
        save_path = output_path.joinpath(f'{speaker_path.name}_text_duplicates.tsv')
        duplicates_df.to_csv(save_path, sep='\t', index=True, quoting=csv.QUOTE_NONE)
        print('⚠ saved to:', save_path)

    return duplicates_df

def time_to_float(time):
    (h, m, s) = time.split(':')
    return np.float(h) * 3600 + np.float(m) * 60 + np.float(s)

def get_audio_by_idx(df, idx):
    row = df.loc[idx]
    ass_file = speaker_path.joinpath(row.ass_file)
    audio_file = ass_file.parent.joinpath(row.audio_file)
    start = time_to_float(row.start)
    end = time_to_float(row.end)
    audio, sr = sf.read(audio_file)
    audio = audio[int(start*sr): int(end*sr)]
    return audio, sr

In [27]:
# путь к корпусам диктора с ass файлами
speaker_path = Path(r'C:\Users\yamilova_da\Desktop\копирование\Дроздов_10')
# путь для сохраенения таблицы с дубликатами
output_path = Path(r'C:\Users\yamilova_da\Desktop')
# ищем дубликаты
# dupl_df = find_text_duplicates(speaker_path, output_path)
dupl_df = find_text_duplicates(speaker_path)

pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 1000)
# кол-во столбцов для отображения
pd.set_option('display.max_columns', None)
# кол-во строк для отображения, выставляем None, чтобы выключить ограничение
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_rows', 20)

# отобразить датафрейм
display(dupl_df)


found 25 text duplicates.


,ass_file,audio_file,pos,text,start,end
0,tender_10_00_master_1.ass,tender_10_00_master_1.wav,16,иди' к на'м жи'ть!,0:01:39.87,0:01:40.75
1,tender_10_01_master_1.ass,tender_10_01_master_1.wav,102,иди' к на'м жи'ть!,0:08:27.16,0:08:28.00
2,tender_10_01_master_1.ass,tender_10_01_master_1.wav,36,я' мы'шка-нору'шка.,0:02:55.79,0:02:56.98
3,tender_10_02_master_1.ass,tender_10_02_master_1.wav,104,я' мы'шка-нору'шка.,0:08:57.25,0:08:58.57
4,tender_10_03_master_1.ass,tender_10_03_master_1.wav,182,я' мы'шка-нору'шка.,0:15:35.32,0:15:36.65
5,tender_10_00_master_1.ass,tender_10_00_master_1.wav,165,колесо' заверте'лось.,0:14:04.64,0:14:05.75
6,tender_10_02_master_1.ass,tender_10_02_master_1.wav,40,колесо' заверте'лось.,0:03:32.50,0:03:33.66
7,tender_10_01_master_1.ass,tender_10_01_master_1.wav,149,кто' в те'реме живё'т?,0:12:10.77,0:12:11.91
8,tender_10_03_master_1.ass,tender_10_03_master_1.wav,160,кто' в те'реме живё'т?,0:13:40.69,0:13:41.75
9,tender_10_00_master_1.ass,tender_10_00_master_1.wav,122,я' лягу'шка-кваку'шка.,0:10:26.49,0:10:27.78


In [34]:
# индексы из датафрейма
indexes = [21,22,23,24]
for idx in indexes:
    audio, sr = get_audio_by_idx(dupl_df, idx)
    print(idx, dupl_df.iloc[idx].text)
    ipd.display(ipd.Audio(audio, rate=sr))


21 семья' бе'з дете'й, что' цвето'к бе'з за'паха.


22 семья' бе'з дете'й, что' цвето'к бе'з за'паха.


23 и' никаки'х ночны'х препира'тельств с со'вестью.


24 и' никаки'х ночны'х препира'тельств с со'вестью.


In [35]:
# указываем индексы строк из датафрейма, которые необходимо закомментировать 
indexes_to_comment = [22,24]

for idx in indexes_to_comment:
    row = dupl_df.iloc[idx]
    ass_file = speaker_path.joinpath(row.ass_file)
    subs = pysubs2.load(ass_file)
    subs[row.pos].type = 'Comment'
    subs.save(ass_file)
    print(f'Файл {ass_file.relative_to(speaker_path)} обновлён.')

Файл tender_4_11_master_1.ass обновлён.
Файл Tender_6_01_master_1.ass обновлён.


In [251]:
# указываем индексы строк из датафрейма, которые необходимо раcкомментировать 
indexes_to_uncomment = [365]

for idx in indexes_to_uncomment:
    row = dupl_df.iloc[idx]
    ass_file = speaker_path.joinpath(row.ass_file)
    subs = pysubs2.load(ass_file)
    subs[row.pos].type = 'Dialogue'
    subs.save(ass_file)
    print(f'Файл {ass_file.relative_to(speaker_path)} обновлён.')

Файл short_set_3_master_1.ass обновлён.
